In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 8.1 Hierarchical Indexing

In [2]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.929616
   2    0.316376
   3    0.183919
b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
d  2    0.653177
   3    0.748907
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [3]:
data["b"]

1    0.204560
3    0.567725
dtype: float64

In [4]:
data["b":"c"]

b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
dtype: float64

In [5]:
data.loc[["b", "d"]]

b  1    0.204560
   3    0.567725
d  2    0.653177
   3    0.748907
dtype: float64

In [7]:
data.loc[:, 2] #select all of the values having the value 2 from the second index level

a    0.316376
c    0.964515
d    0.653177
dtype: float64

Hierarchical indexing plays an important role in reshaping data and in group-based operations like forming a pivot table.

In [8]:
data

a  1    0.929616
   2    0.316376
   3    0.183919
b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
d  2    0.653177
   3    0.748907
dtype: float64

In [9]:
data.unstack() # unstack the inner-level by default of the row-index

,1,2,3
a,0.929616,0.316376,0.183919
b,0.204560,NaN,0.567725
c,0.595545,0.964515,NaN
d,NaN,0.653177,0.748907


In [10]:
data.unstack().stack()

a  1    0.929616
   2    0.316376
   3    0.183919
b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
d  2    0.653177
   3    0.748907
dtype: float64

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

These names supersede the `name` attribute, which is used only with single-level indexes.

In [14]:
frame.index.nlevels

2

In [15]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

A MultiIndex can be created by itself and then reused; the columns in the preceding DataFrame with level names could also be created like this:
```
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green", "Red", "Green"]],
                          names=["state", "color"])
```

### Reordering and Sorting Levels

The `swaplevel` method takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):

In [16]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` by default sorts the data lexicographically using all the index levels, but you can choose to use only a single level or a subset of levels to sort by passing the level argument. For example:

In [17]:
frame.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           names=['key1', 'key2'])

In [18]:
frame.index[0]

('a', 1)

In [23]:
frame.sort_index(level=1) # level 0 is the level of key1. sort_index sort by rows

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [19]:
frame.swaplevel(0, 1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [26]:

frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

:::{.callout-note}
Data selection performance is much better on hierarchically indexed objects if the index is lexicographically sorted starting with the outermost level—that is, the result of calling `sort_index(level=0)` or `sort_index()`.

### Summary Statistics by Level

In [20]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Many descriptive and summary statistics on DataFrame and Series have a level option in which you can specify the `level` you want to aggregate by on a particular axis. 

In [21]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [22]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [23]:

frame.groupby(level="color", axis="columns").sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexing with a DataFrame's columns

In [24]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [26]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

DataFrame’s `set_index` function will create a new DataFrame using one or more of its columns as the index:

In [27]:
frame.set_index(["c", "d"], drop=False) #keeps the columns

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index`, on the other hand, does the opposite of `set_index`; the hierarchical index levels are moved into the columns:

In [28]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [30]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combining and Merging Datasets

Data contained in pandas objects can be combined in a number of ways:

`pandas.merge`
Connect rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database `join` operations.

`pandas.concat`
Concatenate or "stack" objects together along an axis.

`combine_first`
Splice together overlapping data to fill in missing values in one object with values from another.

### Database-Style DataFrame Joins

In [31]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")}) #note dtype="Int64". This will allow nullable integers. 
df1


,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [32]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


This is an example of a many-to-one join

In [33]:
pd.merge(df1, df2) #use overlapping column names (inner join) as keys to join. Note c in df1 and d in df2 are not included. 

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [39]:
pd.merge(df1, df2, on="key") #same result, recommended approach

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [34]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [35]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [37]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


Table 8.1: Different join types with the how argument
Option|	Behavior
|:----------|:--------------------------------------------------------|
how="inner"|	Use only the key combinations observed in both tables
how="left"|	Use all key combinations found in the left table
how="right"|	Use all key combinations found in the right table
how="outer"	|Use all key combinations observed in both tables together

By default, pandas.merge does an "inner" join; the keys in the result are the intersection, or the common set found in both tables. Other possible options are "left", "right", and "outer". The outer join takes the union of the keys, combining the effect of applying both left and right joins:

In [38]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


In [39]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [45]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})
df1


,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [46]:
df2


,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


Many-to-many merges form the Cartesian product of the matching keys. Here’s an example:

In [47]:
pd.merge(df1, df2, on="key", how="left") #keep all the keys of the left dataframe

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [48]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [49]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
left


,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [50]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [51]:
pd.merge(left, right, on=["key1", "key2"], how="outer") # merge with multiple keys

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


:::{.callout-warning}
When you're joining columns on columns, the indexes on the passed DataFrame objects are discarded. If you need to preserve the index values, you can use `reset_index` to append the index to the columns.

In [53]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [54]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [52]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [55]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


Table 8.2: pandas.merge function arguments
Argument	|Description
|:---------|:-----------------------------------------------------------------------|
left|	DataFrame to be merged on the left side.
right|	DataFrame to be merged on the right side.
how	|Type of join to apply: one of "inner", "outer", "left", or "right"; defaults to "inner".
on	|Column names to join on. Must be found in both DataFrame objects. If not specified and no other join keys given, will use the intersection of the column names in left and right as the join keys.
left_on	|Columns in left DataFrame to use as join keys. Can be a single column name or a list of column names.
right_on	|Analogous to left_on for right DataFrame.
left_index|	Use row index in left as its join key (or keys, if a MultiIndex).
right_index|	Analogous to left_index.
sort|	Sort merged data lexicographically by join keys; False by default.
suffixes|	Tuple of string values to append to column names in case of overlap; defaults to ("_x", "_y") (e.g., if "data" in both DataFrame objects, would appear as "data_x" and "data_y" in result).
copy	|If False, avoid copying data into resulting data structure in some exceptional cases; by default always copies.
validate|	Verifies if the merge is of the specified type, whether one-to-one, one-to-many, or many-to-many. See the docstring for full details on the options.
indicator|	Adds a special column _merge that indicates the source of each row; values will be "left_only", "right_only", or "both" based on the origin of the joined data in each row.

### Merging on Index

In some cases, the merge key(s) in a DataFrame will be found in its index (row labels). In this case, you can pass left_index=True or right_index=True (or both) to indicate that the index should be used as the merge key:

In [2]:
import pandas as pd
import numpy as np

In [3]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
left1


,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [4]:
right1


,group_val
a,3.5
b,7.0


In [5]:
pd.merge(left1, right1, left_on="key", right_index=True) 
# note the index of left1 was preserved in "many to one" merge in this case

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [6]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [7]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})
lefth


,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [8]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [9]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)


,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [10]:
pd.merge(lefth, righth, left_on=["key1", "key2"],
         right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


In [11]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")
left2


,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [12]:
right2


,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [13]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [14]:
left2.join(right2, how="outer") # join method using index from both dataframes

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [15]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [16]:
right1

,group_val
a,3.5
b,7.0


In [17]:
left1.join(right1, on="key") #perform left-join on "key" of left1 with index on right1

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [18]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
another


,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [20]:
left2, right2

(   Ohio  Nevada
 a     1       2
 c     3       4
 e     5       6,
    Missouri  Alabama
 b         7        8
 c         9       10
 d        11       12
 e        13       14)

for simple index-on-index merges, you can pass a list of DataFrames to join as an alternative to using the more general pandas.concat function described in the next section:

In [19]:
left2.join([right2, another]) #left join


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [21]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


In [22]:
arr = np.arange(12).reshape((3, 4))
arr


array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [23]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [25]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")
s1

a    0
b    1
dtype: Int64

In [26]:
s2


c    2
d    3
e    4
dtype: Int64

In [27]:
s3


f    5
g    6
dtype: Int64

In [28]:
pd.concat([s1, s2, s3]) #Note concat use index to align data

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

By default, pandas.concat works along axis="index", producing another Series. If you pass axis="columns", the result will instead be a DataFrame:

In [29]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [30]:
s4 = pd.concat([s1, s3])
s4


a    0
b    1
f    5
g    6
dtype: Int64

In [31]:
pd.concat([s1, s4], axis="columns")


,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [32]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


In [34]:
s1,s2,s3

(a    0
 b    1
 dtype: Int64,
 c    2
 d    3
 e    4
 dtype: Int64,
 f    5
 g    6
 dtype: Int64)

A potential issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the keys argument:

In [35]:
pd.concat([s1, s1, s3])#, keys=["one", "two", "three"])

a    0
b    1
a    0
b    1
f    5
g    6
dtype: Int64

In [33]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"]) # keys corresponding to each series
result


one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

When you call result.unstack(), the operation will unstack the innermost index, promoting it to the DataFrame's columns. 

In [36]:
result.unstack() # unstack the innermost index to columns

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


In [37]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [38]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
df1


,one,two
a,0,1
b,2,3
c,4,5


In [39]:
df2


,three,four
a,5,6
c,7,8


In [40]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"]) #left join on index

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [41]:
pd.concat({"level1": df1, "level2": df2}, axis="columns") #alternatively

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [42]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
          names=["upper", "lower"]) #name the created axis levels with the names argument

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [43]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])
df1


,a,b,c,d
0,0.313668,-0.464684,-0.729166,-0.483910
1,0.153606,-2.262991,0.433413,-0.417137
2,-0.230394,0.275671,1.247662,0.943670


In [44]:
df2

,b,d,a
0,-1.241297,0.440639,1.613193
1,0.202803,-0.717498,0.807893


 you can pass ignore_index=True, which discards the indexes from each DataFrame and concatenates the data in the columns only, assigning a new default index:

In [45]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.313668,-0.464684,-0.729166,-0.483910
1,0.153606,-2.262991,0.433413,-0.417137
2,-0.230394,0.275671,1.247662,0.943670
3,1.613193,-1.241297,NaN,0.440639
4,0.807893,0.202803,NaN,-0.717498


Table 8.3: pandas.concat function arguments
Argument|	Description
:-----------------|:----------------------------------------------------------------------|
objs|	List or dictionary of pandas objects to be concatenated; this is the only required argument
axis|	Axis to concatenate along; defaults to concatenating along rows (axis="index")
join|	Either "inner" or "outer" ("outer" by default); whether to intersect (inner) or union (outer) indexes along the other axes
keys|	Values to associate with objects being concatenated, forming a hierarchical index along the concatenation axis; can be a list or array of arbitrary values, an array of tuples, or a list of arrays (if multiple-level arrays passed in levels)
levels|	Specific indexes to use as hierarchical index level or levels if keys passed
names|	Names for created hierarchical levels if keys and/or levels passed
verify_integrity|	Check new axis in concatenated object for duplicates and raise an exception if so; by default (False) allows duplicates
ignore_index|	Do not preserve indexes along concatenation axis, instead produce a new range(total_length) index

### Combining Data with Overlap

In [46]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])
a


f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [47]:
b


a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [48]:
np.where(pd.isna(a), b, a) #performs the array-oriented equivalent of an if-else expression:
# NOte index are not aligned. 

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

Using numpy.where does not check whether the index labels are aligned or not (and does not even require the objects to be the same length), so if you want to line up values by index, use the Series combine_first method:

If a value is present in `a` at a specific index or column and is NaN or missing, it will be filled with the corresponding value from `b` at that index or column. If the value in a is not missing, it remains unchanged.

In [49]:
a.combine_first(b) # note the index are sorted

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [50]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df1


,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [51]:
df2


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [52]:
#With DataFrames, combine_first does the same thing column by column, 
df1.combine_first(df2) #The output of combine_first with DataFrame objects will have the union of all the column names.

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 Reshaping and Pivoting

In [53]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


### Reshaping with Hierarchical Indexing

stack

This “rotates” or pivots from the columns in the data to the rows.

unstack

This pivots from the rows into the columns.

In [54]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [55]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


By default, the innermost level is unstacked (same with stack). You can unstack a different level by passing a level number or name:

In [56]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [57]:
result.unstack(level=0)


state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [58]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [59]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
s1

a    0
b    1
c    2
d    3
dtype: Int64

In [60]:
s2

c    4
d    5
e    6
dtype: Int64

In [61]:
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [62]:
data2.unstack()


,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [63]:
data2.unstack().stack() #stacking filters out missing data by default, so the operation is more easily invertible:


one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [64]:
data2.unstack().stack(dropna=False)

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [66]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [65]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df


side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [67]:
df.unstack(level="state") #When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result:

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

As with unstack, when calling stack we can indicate the name of the axis to stack:

In [68]:
df.unstack(level="state").stack(level="side")

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

### Pivoting “Long” to “Wide” Format

In [69]:
data = pd.read_csv("examples/macrodata.csv")
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2008,3,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
199,2008,4,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
200,2009,1,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71
201,2009,2,12901.504,9189.0,1456.678,1023.528,10077.5,214.469,1653.6,0.18,9.2,307.226,3.37,-3.19


In [70]:
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]


In [71]:
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


pop method on the DataFrame, which returns a column while deleting it from the DataFrame at the same time.

pandas.PeriodIndex (which represents time intervals rather than points in time), discussed in more detail in Ch 11: Time Series,

In [72]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date") 
periods


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [73]:
data.index = periods.to_timestamp("D") 
#The "D" argument passed to to_timestamp represents day frequency. 
# It is used to create a Timestamp that represents a specific day at midnight. ex: 1959-01-01 00:00:00
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [74]:
data = data.reindex(columns=["realgdp", "infl", "unemp"]) #redindex reorder the columns, but here do nother because the same order of columns
data.head()


,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [75]:
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [76]:
data.stack()

date        item   
1959-01-01  realgdp     2710.349
            infl           0.000
            unemp          5.800
1959-04-01  realgdp     2778.801
            infl           2.340
                         ...    
2009-04-01  infl           3.370
            unemp          9.200
2009-07-01  realgdp    12990.341
            infl           3.560
            unemp          9.600
Length: 609, dtype: float64

In [77]:
data.stack().reset_index()

,date,item,0
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


In [78]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

In [79]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [80]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [81]:
long_data.index.name = None

In [82]:
long_data

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


In [83]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,-0.094225
1,1959-01-01,infl,0.000,-1.944494
2,1959-01-01,unemp,5.800,0.251271
3,1959-04-01,realgdp,2778.801,-1.157011
4,1959-04-01,infl,2.340,2.017680
5,1959-04-01,unemp,5.100,1.337269
6,1959-07-01,realgdp,2775.488,1.299282
7,1959-07-01,infl,2.740,0.744027
8,1959-07-01,unemp,5.300,-0.968294
9,1959-10-01,realgdp,2785.204,-0.938596


By omitting the last argument, you obtain a DataFrame with hierarchical columns:

In [84]:
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()


value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -1.944494 -0.094225  0.251271
1959-04-01  2.34  2778.801   5.1  2.017680 -1.157011  1.337269
1959-07-01  2.74  2775.488   5.3  0.744027  1.299282 -0.968294
1959-10-01  0.27  2785.204   5.6 -1.831547 -0.938596 -0.414471
1960-01-01  2.31  2847.699   5.2  1.109741  0.484895 -3.292301

In [85]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


Note that pivot is equivalent to creating a hierarchical index using set_index followed by a call to unstack:

In [86]:
long_data.set_index(["date", "item"])

value    value2
date       item                        
1959-01-01 realgdp   2710.349 -0.094225
           infl         0.000 -1.944494
           unemp        5.800  0.251271
1959-04-01 realgdp   2778.801 -1.157011
           infl         2.340  2.017680
...                       ...       ...
2009-04-01 infl         3.370  0.835688
           unemp        9.200 -1.372945
2009-07-01 realgdp  12990.341  0.484169
           infl         3.560  0.794406
           unemp        9.600 -0.405883

[609 rows x 2 columns]

In [87]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -1.944494 -0.094225  0.251271
1959-04-01  2.34  2778.801   5.1  2.017680 -1.157011  1.337269
1959-07-01  2.74  2775.488   5.3  0.744027  1.299282 -0.968294
1959-10-01  0.27  2785.204   5.6 -1.831547 -0.938596 -0.414471
1960-01-01  2.31  2847.699   5.2  1.109741  0.484895 -3.292301

### Pivoting “Wide” to “Long” Format

In [88]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


 `pandas.melt` merges multiple columns into one, producing a DataFrame that is longer than the input. The "key" column may be a group indicator, and the other columns are data values. When using pandas.melt, we must indicate which columns (if any) are group indicators. Let's use "key" as the only group indicator here:

In [89]:
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


Using pivot, we can reshape back to the original layout:

In [90]:
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [91]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [93]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [92]:
#You can also specify a subset of columns to use as value columns:
pd.melt(df, id_vars="key", value_vars=["A", "B"]) #col C is not chosen

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [95]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [94]:
pd.melt(df, value_vars=["A", "B", "C"]) # no group identifier


,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [96]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
